In [1]:
#import pandas
import pandas as pd

In [2]:
#Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

#Other imports
import os 
import wget
import re
from selenium.webdriver.common.keys import Keys
import time




In [3]:
# A classe Service é usada para iniciar uma instancia do chrome webdriver
service = Service()

# Webdriver.ChromeOptions é usado para definir a preferencia para o browser chrome
options = webdriver.ChromeOptions()

# Inicia-se a instancia do Chrome WebDriver com as 'options' e 'service'
driver = webdriver.Chrome(service=service, options=options)

In [4]:
#navegando ate o insta
link = "https://www.instagram.com/"
driver.get(link)

In [5]:
#colocando email e senha
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
password.clear()
username.send_keys('bfmari')
password.send_keys('xuxubeleza8')

In [6]:
#clickar para logar
log_in = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [7]:
agora_Nao = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Agora não')]"))).click()
agora_Nao2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Agora não')]"))).click()

In [10]:
#entrar no perfil
profile = driver.find_elements(By.CLASS_NAME, 'xpdipgo')
profile[0].click()
    

In [85]:
# Constantes para os seletores do Selenium
TIPO_SELECTOR = "xbkimgs"
DATA_SELECTOR = "x1p4m5qa"
CURTIDAS_SELECTOR_1 = "html-span"
CURTIDAS_SELECTOR_2 = "xvs91rp"
COMENTARIOS_SELECTOR_1 = "_a9zs"
COMENTARIOS_SELECTOR_2 = "_a9ym"
TAGS_SELECTOR = "_a9z5"


#listas vazias para armazenar os dados coletados
tipos = []
datas = []
curtidas = []
quantidades_comentarios = []
comentarios = []
tags = []
carrosseis = []
interacoes = []


# Criar um conjunto para armazenar os links únicos das publicações
links_unicos = set()

# Simule a rolagem da página em pequenos incrementos
for i in range(2):  # Altere o número de vezes conforme necessário
    driver.execute_script("window.scrollBy(0, 500);")  # Role a página em 500 pixels
    time.sleep(1)  # Aguarde um momento para a página carregar

    # Agora tente encontrar os elementos "tipo" após a rolagem
    divs_tipo = driver.find_elements(By.CLASS_NAME, "xbkimgs")
    for div_tipo in divs_tipo:
        a = div_tipo.find_element(By.TAG_NAME, "a")
        href = a.get_attribute("href")

        # Verifique se o link único já foi coletado anteriormente
        if href not in links_unicos:
            links_unicos.add(href)
            # Agora pegue o "tipo" da publicação
            if div_tipo.find_elements(By.CLASS_NAME, "_aatp"):
                svg = div_tipo.find_element(By.CLASS_NAME, "_aatp")
                try:
                    title = svg.find_element(By.CLASS_NAME, "x9bdzbf")
                    aria_label = title.get_attribute("aria-label")
                    tipos.append(aria_label)  # Adicione o tipo à lista
                except NoSuchElementException:
                    print("Nenhum elemento 'x9bdzbf' encontrado - pode ser uma foto")
            else:
                aria_label = "Foto"
                tipos.append(aria_label)
                

#começando a percorrer para pegar data e like
container_feed = driver.find_elements(By.CLASS_NAME, "_aagu")
for elemento in container_feed:
    try:
        #data
        elemento.click()
        time.sleep(0.3)  # Aguarde um momento para a página carregar completamente
        data = driver.find_element(By.CLASS_NAME, "x1p4m5qa").get_attribute('title')
        datas.append(data) # Adiciona a data à lista
        
        #like
        like = driver.find_elements(By.CLASS_NAME, "html-span")
        like2 = driver.find_elements(By.CLASS_NAME, "xvs91rp")
        if like:
            if len(like) > 3:
                try:
                    ultimo_elemento = like[3]
                    quantidade_likes = int(ultimo_elemento.text) 
                    curtidas.append(quantidade_likes) # Adiciona a quantidade de curtidas à lista
                except:
                    curtidas.append(0) # Adiciona 0 à lista se não houver curtidas
        if like2:
            if len(like2) > 5:
                span = like2[5]
                span.click()
                time.sleep(0.5)  # Aguarde um momento para a página carregar completamente
                elemento = driver.find_elements(By.CLASS_NAME, "html-span")[3].text
                quantidade_likes = int(elemento)
                curtidas.append(quantidade_likes) # Adiciona a quantidade de curtidas à lista
            else:
                curtidas.append(0) # Adiciona 0 à lista se não houver curtidas
        else:
            print("Nenhum elemento com a classe 'html-span' encontrado")

        
            
        #comentarios
        
        comments = driver.find_elements(By.CLASS_NAME, "_a9zs")
        qtd_comments = driver.find_elements(By.CLASS_NAME, "_a9ym")
        comentarios_validos = [re.sub(r'\W+', ' ', elemento.text).strip() for elemento in comments if '#' not in elemento.text][1:]
        comentarios_validos = [comentario for comentario in comentarios_validos if comentario]  # Remover comentários vazios
        quantidade_comentarios = len(qtd_comments)
        quantidades_comentarios.append(quantidade_comentarios)  # Adiciona a quantidade de comentários à nova lista
        comentarios.append(comentarios_validos)  # Adiciona os comentários à lista

        #TAG 

        tag_elements = driver.find_elements(By.CLASS_NAME, "_a9z5")
        if tag_elements:
            tag = tag_elements[0]
            texto_completo = tag.text
            hashtags = re.findall(r'#\w+', texto_completo)
            hashtags_como_string = ' '.join(hashtags)
            tags.append(hashtags_como_string)  # Adiciona as hashtags à lista
        else:
            tags.append("null")  # Adiciona uma string vazia à lista se não houver hashtags

        #carroussel 

        carrousel = driver.find_elements(By.CLASS_NAME, "_9zm2")
        if carrousel:
            carrousel = 1
        else:
            carrousel = 0
        carrosseis.append(carrousel)  # Adiciona a informação sobre o carrossel à lista

        #interaçoes

        interacao = quantidade_likes + quantidade_comentarios
        interacoes.append(interacao)  # Adiciona a quantidade de interações à lista

    except StaleElementReferenceException:
        print("Elemento não está mais anexado ao DOM. Lidando com a exceção...")
        continue
    finally:
        driver.back()

dados = {
    'Tipo': tipos,
    'Data': datas,
    'Curtidas': curtidas,
    'Qtd_Comentarios': quantidades_comentarios,
    'Comentarios': comentarios,
    'Tags': tags,
    'Carrossel': carrosseis,
    'Interacoes': interacoes
}

df = pd.DataFrame(dados)

# Salve o DataFrame em um arquivo Excel
df.to_excel('dados_instagram.xlsx', index=False)

ValueError: All arrays must be of the same length

In [51]:
#listas vazias para armazenar os dados coletados
tipos = []
datas = []
curtidas = []
quantidades_comentarios = []
comentarios = []
tags = []
carrosseis = []
interacoes = []


# Criar um conjunto para armazenar os links únicos das publicações
links_unicos = set()

def collect_tipo(driver):

    # Agora tente encontrar os elementos "tipo" após a rolagem
    divs_tipo = driver.find_elements(By.CLASS_NAME, "xbkimgs")
    for div_tipo in divs_tipo:
        a = div_tipo.find_element(By.TAG_NAME, "a")
        href = a.get_attribute("href")

        # Verifique se o link único já foi coletado anteriormente
        if href not in links_unicos:
            links_unicos.add(href)
            # Agora pegue o "tipo" da publicação
            if div_tipo.find_elements(By.CLASS_NAME, "_aatp"):
                svg = div_tipo.find_element(By.CLASS_NAME, "_aatp")
                try:
                    title = svg.find_element(By.CLASS_NAME, "x9bdzbf")
                    aria_label = title.get_attribute("aria-label")
                    tipos.append(aria_label)  # Adicione o tipo à lista
                except NoSuchElementException:
                    tipos.append("Foto")  # Adiciona 'Foto' como valor padrão
            else:
                tipos.append("Foto")  # Adiciona 'Foto' como valor padrão

    return tipos
    
def collect_data_and_likes(elemento):
    data = None
    quantidade_likes = 0

    try:
        # Data
        elemento.click()
        time.sleep(0.5)  # Aguarde um momento para a página carregar completamente
        data = driver.find_element(By.CLASS_NAME, "x1p4m5qa").get_attribute('title')

        #like
        like = driver.find_elements(By.CLASS_NAME, "html-span")
        like2 = driver.find_elements(By.CLASS_NAME, "xvs91rp")
        
        if len(like) > 3:
            try:
                ultimo_elemento = like[3].text
                quantidade_likes = int(ultimo_elemento) + 1 
            except ValueError:
                print("Erro ao converter o texto para número")
                quantidade_likes = 0
        else:
            if len(like2) > 5:
                span = like2[5]
                span.click()
                time.sleep(0.5)  # Aguarde um momento para a página carregar completamente
                elemento = driver.find_elements(By.CLASS_NAME, "html-span")[3].text
                quantidade_likes = int(elemento)
            else:
                quantidade_likes = 0
    except NoSuchElementException:
        data = "Sem data"
        quantidade_likes = 0

    return data, quantidade_likes

        
            
def collect_comments_and_count(elemento):
    comentarios_validos = []
    quantidade_comentarios = 0

    try:
        time.sleep(0.2)  # Aguarde um momento para a página carregar completamente
        # Comentários
        comments = driver.find_elements(By.CLASS_NAME, "_a9zs")
        qtd_comments = driver.find_elements(By.CLASS_NAME, "_a9ym")
        comentarios_validos = [re.sub(r'\W+', ' ', elemento.text).strip() for elemento in comments if '#' not in elemento.text][1:]
        comentarios_validos = [comentario for comentario in comentarios_validos if comentario]  # Remover comentários vazios
        quantidade_comentarios = len(qtd_comments)
    except:
        comentarios_validos = ["Sem comentários"]
        quantidade_comentarios = 0

    return comentarios_validos, quantidade_comentarios

def collect_tags(elemento):
    tag_elements = driver.find_elements(By.CLASS_NAME, "_a9z5")
    if tag_elements:
        tag = tag_elements[0]
        texto_completo = tag.text
        hashtags = re.findall(r'#\w+', texto_completo)
        hashtags_como_string = ' '.join(hashtags)
        return hashtags_como_string
    else:
        return "null"

def collect_carousel(elemento):
    carousel = driver.find_elements(By.CLASS_NAME, "_9zm2")
    if carousel:
        return 'sim'
    else:
        return 'não'

def collect_interactions(quantidade_likes, quantidade_comentarios):
    interacao = quantidade_likes + quantidade_comentarios
    return interacao
    
##----APP----##

# Simule a rolagem da página em pequenos incrementos
for i in range(2):  # Altere o número de vezes conforme necessário
    driver.execute_script("window.scrollBy(0, 500);")  # Role a página em 500 pixels
    time.sleep(1)  # Aguarde um momento para a página carregar

# Coleta do tipo
tipos = collect_tipo(driver)

container_feed = driver.find_elements(By.CLASS_NAME, "_aagu")
for elemento in container_feed:
    try: 
        # Coleta de data e likes
        data, quantidade_likes = collect_data_and_likes(elemento)
        datas.append(data)
        curtidas.append(quantidade_likes)
    
        # Coleta de comentários e contagem
        comentarios_validos, quantidade_comentarios = collect_comments_and_count(elemento)
        comentarios.append(comentarios_validos)
        quantidades_comentarios.append(quantidade_comentarios)
    
        # Coleta de tags
        tag_string = collect_tags(elemento)
        tags.append(tag_string)
    
        # Coleta de carrossel
        carousel_value = collect_carousel(elemento)
        carrosseis.append(carousel_value)
    
        # Coleta de interações
        interacao = collect_interactions(quantidade_likes, quantidade_comentarios)
        interacoes.append(interacao)
    except StaleElementReferenceException:
        print("Elemento não está mais anexado ao DOM. Lidando com a exceção...")
        continue
    finally:
        driver.back()

print("Tipos:", tipos)
print("Datas:", datas)
print("Curtidas:", curtidas)
print("Quantidades de comentários:", quantidades_comentarios)
print("Comentários:", comentarios)
print("Tags:", tags)
print("Carrosseis:", carrosseis)
print("Interações:", interacoes)

    
dados = {
    'Tipo': tipos,
    'Data': datas,
    'Curtidas': curtidas,
    'Qtd_Comentarios': quantidades_comentarios,
    'Comentarios': comentarios,
    'Tags': tags,
    'Carrossel': carrosseis,
    'Interacoes': interacoes
}

df = pd.DataFrame(dados)

# Salve o DataFrame em um arquivo Excel
df.to_excel('dados_instagram.xlsx', index=False)

Tipos: ['Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Carrossel', 'Vídeo', 'Foto', 'Foto', 'Carrossel', 'Foto', 'Foto', 'Foto', 'Foto', 'Carrossel', 'Carrossel', 'Foto', 'Carrossel']
Datas: ['28 de janeiro de 2024', '20 de janeiro de 2024', '19 de janeiro de 2024', '15 de novembro de 2023', '15 de outubro de 2023', '3 de agosto de 2023', '21 de fevereiro de 2023', '15 de fevereiro de 2023', '13 de janeiro de 2023', '25 de dezembro de 2022', '8 de agosto de 2022', '30 de outubro de 2021', '28 de janeiro de 2021', '20 de novembro de 2020', '18 de setembro de 2020', '10 de setembro de 2020', '15 de agosto de 2020', '17 de maio de 2020', '20 de abril de 2020', '16 de março de 2020', '6 de março de 2020', '13 de fevereiro de 2020', '11 de janeiro de 2020', '21 de janeiro de 2019']
Curtidas: [44, 24, 58, 59, 59, 62, 79, 69, 36, 77, 58, 57, 310, 36, 22, 80, 75, 67, 51, 19, 40, 28, 68, 75]
Quantid

## Salvando as imagens no computador


In [10]:
keyword = "@valim.g"
path = os.getcwd()
path = os.path.join(path, keyword[1:])

os.mkdir(path)
path

'C:\\Users\\Gabriel\\Downloads\\Crawler\\valim.g'

In [11]:
counter = 0
for image in images:
    save_as = os.path.join(path, keyword[1:] + str(counter) + '.jpg')
    wget.download(image, save_as)
    counter += 1

100% [..............................................................................] 99935 / 99935

In [18]:
for elemento in lista_posts:
    link = elemento.find_element(By.TAG_NAME, 'a').get_attribute('href')
    print(link)
    

NameError: name 'lista_posts' is not defined